In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from app.data_processing.utils import *

In [2]:
import pandas as pd
import pattern.es as es
import regex as re

In [5]:
fr = pd.read_csv('../data/peso_frutas.csv')
vr = pd.read_csv('../data/peso_hortalizas.csv')
bedca = pd.read_csv('../data/bedca.csv',index_col=0)

In [24]:
bedca_ingredients = bedca["nombre"].tolist()
bedca_ingredients = [x.lower() for x in bedca_ingredients]
bedca_ingredients = ingredients_to_plural(bedca_ingredients) 

In [14]:
fr = fr.drop('Peso bruto (g)',axis=1)

In [15]:
vg = fr.append(vr)

In [16]:
vg['Alimento'] = vg['Alimento'].apply(es.singularize)

In [22]:
def get_numeric_value(row):
    numbers = [word for word in row.split() if word.isdigit()]
    if len(numbers) > 0:
        return int(numbers[0])
    return None

def set_bedca_name(row):
    ext = extract_ingredient(row,bedca_ingredients)
    if ext != "None":
        return ext
    else:
        return row

In [25]:
vg['Numero'] = vg['Medida'].apply(get_numeric_value)
vg['Alimento'] = vg['Alimento'].apply(set_bedca_name)
vg['Peso x Unidad'] = vg['Peso neto (g)'].divide(vg['Numero'])
vg.rename(columns={'Alimento': 'Ingrediente'}, inplace=True)

In [12]:
vg.to_csv('../data/pesos_vegetales.csv', index=False)

In [21]:
recipes = pd.read_csv('../data/output.csv', sep='|')

In [25]:
preferences = {'Carne': 1.05, 
'Pollo': 0.3333333333333333, 
'Pescado': 0.3666666666666667, 
'Legumbres': 0.8833333333333334, 
'Verduras': -0.4, 'Patatas': 0.275, 'Arroz': 0.0, 'Pasta': 0.25, 
'Huevos': -0.5833333333333333}

In [34]:
ing = pd.read_csv('../data/ingredients_100.csv')

In [114]:
recipes = pd.read_csv('../data/filtered_recipes_100.csv', sep='|')

In [55]:
recipes = recipes.head(8)

In [115]:
recipes

,Recipe_id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes,parse_rate,total_ingredients,parsed_ingredients
0,74154,Recetas de Aperitivos y tapas,Receta de burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,0,0,2020-03-25,"10 tortillas de harina para burritos,500 gramo...",1.0,7.0,7.0
1,74146,Recetas de Aperitivos y tapas,Receta de croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,0,0,2020-03-24,"1 pechuga de pollo cocida y troceada,1 pellizc...",1.0,11.0,11.0
2,74056,Recetas de Aperitivos y tapas,Receta de tomates cherry confitados,NaN,baja,6.0,1h 30m,Acompañamiento,https://www.recetasgratis.net/receta-de-tomate...,0,0,2020-02-26,"500 gramos de tomates cherry,1 rama de romero ...",1.0,9.0,9.0
3,74043,Recetas de Aperitivos y tapas,Receta de edamame,NaN,muy baja,4.0,10m,Entrante,https://www.recetasgratis.net/receta-de-edamam...,0,0,2020-02-18,"1 bolsa de edamame (vainas de soja),1 pellizco...",1.0,5.0,5.0
4,73727,Recetas de Aperitivos y tapas,Receta de bechamel para croquetas,NaN,baja,3.0,15m,NaN,https://www.recetasgratis.net/receta-de-becham...,0,0,2020-02-04,"1 litro de leche,50 gramos de mantequilla,1 ce...",1.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7239,6618,Recetas de Verduras,Receta de mangú dominicano,4.3,baja,2.0,45m,Acompañamiento,https://www.recetasgratis.net/receta-de-mangu-...,33,4,NaN,"6 unidades de plátanos verdes,4 tazas de agua ...",1.0,7.0,7.0
7240,31007,Recetas de Verduras,Receta de pastel de berenjenas y pimientos con...,3.8,media,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-pastel...,2,8,NaN,"400 gramos de berenjenas enteras,200 gramos de...",1.0,6.0,6.0
7241,6371,Recetas de Verduras,Receta de pimientos verdes con arroz y nueces,4.6,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-pimien...,1,6,NaN,"4 pimientos verdes medianos,1 cebolla mediana ...",1.0,11.0,11.0
7242,56,Recetas de Verduras,Receta de alcachofas glaseadas,4.0,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-alcach...,0,4,NaN,"6 fondos de alcachofa,300 gr. de agua mineral,...",1.0,7.0,7.0


In [119]:
x = recipes.set_index('Recipe_id', drop=False)
x.index.name = 'index'

In [120]:
x

,Recipe_id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes,parse_rate,total_ingredients,parsed_ingredients
index,,,,,,,,,,,,,,,,
74154,74154,Recetas de Aperitivos y tapas,Receta de burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,0,0,2020-03-25,"10 tortillas de harina para burritos,500 gramo...",1.0,7.0,7.0
74146,74146,Recetas de Aperitivos y tapas,Receta de croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,0,0,2020-03-24,"1 pechuga de pollo cocida y troceada,1 pellizc...",1.0,11.0,11.0
74056,74056,Recetas de Aperitivos y tapas,Receta de tomates cherry confitados,NaN,baja,6.0,1h 30m,Acompañamiento,https://www.recetasgratis.net/receta-de-tomate...,0,0,2020-02-26,"500 gramos de tomates cherry,1 rama de romero ...",1.0,9.0,9.0
74043,74043,Recetas de Aperitivos y tapas,Receta de edamame,NaN,muy baja,4.0,10m,Entrante,https://www.recetasgratis.net/receta-de-edamam...,0,0,2020-02-18,"1 bolsa de edamame (vainas de soja),1 pellizco...",1.0,5.0,5.0
73727,73727,Recetas de Aperitivos y tapas,Receta de bechamel para croquetas,NaN,baja,3.0,15m,NaN,https://www.recetasgratis.net/receta-de-becham...,0,0,2020-02-04,"1 litro de leche,50 gramos de mantequilla,1 ce...",1.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6618,6618,Recetas de Verduras,Receta de mangú dominicano,4.3,baja,2.0,45m,Acompañamiento,https://www.recetasgratis.net/receta-de-mangu-...,33,4,NaN,"6 unidades de plátanos verdes,4 tazas de agua ...",1.0,7.0,7.0
31007,31007,Recetas de Verduras,Receta de pastel de berenjenas y pimientos con...,3.8,media,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-pastel...,2,8,NaN,"400 gramos de berenjenas enteras,200 gramos de...",1.0,6.0,6.0
6371,6371,Recetas de Verduras,Receta de pimientos verdes con arroz y nueces,4.6,NaN,NaN,NaN,NaN,https://www.recetasgratis.net/receta-de-pimien...,1,6,NaN,"4 pimientos verdes medianos,1 cebolla mediana ...",1.0,11.0,11.0


In [113]:
recipes.index.values

array([74154, 74146, 74056, ...,  6371,    56,     5])